In [8]:
from torchvision.datasets import ImageFolder
import basic_torch_function
from torchvision import models
path = "D:\\Casper\\OTHER\\Data\\rock_paper_scissor\\rps-cv-images"
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

# from torchvision.transforms import v2
# transforms = v2.Compose([
#     v2.RandomResizedCrop(size=(224, 224), antialias=True),
#     v2.ToDtype(torch.float32, scale=True),
# ])
transforming = transforms.Compose([
    transforms.ToTensor()
])

dataset = ImageFolder(path, transforming)

In [4]:
data_loaders = basic_torch_function.get_dataloaders(dataset, 0.8, 0.5, 32)

--------- INDEX checking ---------
Original: [0, 1, 2, 3, 4]
Shuffled: [1400, 2046, 1476, 439, 798]
--------- INDEX shuffled ---------

Total number of samples: 2188 datapoints
Number of train samples: 55 batches/ 1750 datapoints
Number of val samples: 7 batches/ 219 datapoints
Number of test samples: 7 batches/ 219 datapoints



In [13]:
model_list ={
    "resnet18": lambda: models.resnet18(),
    "resnet18_mod": lambda: mod_resnet(BasicBlock, [2, 2, 2, 2], channel_num_list=[2, 2, 2, 2, 2], num_classes=3, input_channel=3),
    "resnet18_mod2": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[2, 2, 2, 2, 2], num_classes=3, input_channel=3),
}
model_names = [
    "resnet18_mod2",
]
model_name = model_names[0]
phases = ['train', 'val']
basic_torch_function.train(model_list, model_name, data_loaders, phases)

test resnet18_mod2
model total parameters: 811
learning rate=0.005


100%|██████████| 55/55 [00:05<00:00,  9.97it/s]


Epoch [1/20], phase: train, samples: 1750, Loss: 0.0333, Top-1 Accuracy: 46.97%, Top-3 Accuracy: 100.00%


100%|██████████| 7/7 [00:00<00:00, 16.28it/s]


Epoch [1/20], phase: val, samples: 219, Loss: 0.0332, Top-1 Accuracy: 39.73%, Top-3 Accuracy: 100.00%


...

Epoch [19/20], phase: val, samples: 219, Loss: 0.0345, Top-1 Accuracy: 69.86%, Top-3 Accuracy: 100.00%


100%|██████████| 55/55 [00:05<00:00, 10.21it/s]


Epoch [20/20], phase: train, samples: 1750, Loss: 0.0019, Top-1 Accuracy: 98.40%, Top-3 Accuracy: 100.00%


100%|██████████| 7/7 [00:00<00:00, 11.29it/s]

Epoch [20/20], phase: val, samples: 219, Loss: 0.0122, Top-1 Accuracy: 91.78%, Top-3 Accuracy: 100.00%
Elapsed time: 121.2694149017334 seconds


# 越深越容易不穩定？？

In [9]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = F.relu(out)

        return out
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x
class mod_resnet(nn.Module):
    def __init__(self, block, layers, channel_num_list, num_classes=1000, input_channel=3):
        super(mod_resnet, self).__init__()
        self.in_channels = channel_num_list[0]
        self.conv1 = nn.Conv2d(input_channel, channel_num_list[0], kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_num_list[0])
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, channel_num_list[1], layers[0])
        self.layer2 = self._make_layer(block, channel_num_list[2], layers[1], stride=2)
        self.layer3 = self._make_layer(block, channel_num_list[3], layers[2], stride=2)
        self.layer4 = self._make_layer(block, channel_num_list[4], layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(channel_num_list[-1] * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x